In [1]:
#Add needed imports
import numpy as np
import pandas as pd
import os
import shap
from numpy import mean
from imblearn.over_sampling import SMOTENC
from sklearn.preprocessing import RobustScaler
from xgboost import XGBClassifier
from sklearn.tree import DecisionTreeClassifier  
from sklearn.model_selection import StratifiedKFold
import tensorflow as tf
from tensorflow import keras
import numpy as np
import keras 
import pydot
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import RobustScaler
from tensorflow.keras.layers import Input, Dense, Activation,Dropout
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras import layers
from keras.utils import plot_model
from imblearn.over_sampling import SMOTENC
from sklearn.metrics import confusion_matrix, accuracy_score, f1_score, precision_score, recall_score
from sklearn.model_selection import StratifiedKFold
import os
import shap

from tensorflow.compat.v1.keras.backend import get_session
tf.compat.v1.disable_v2_behavior()
#Read data
proccessed_data_path =os.path.join(os.path.pardir,os.path.pardir,'data','processed')
train_path = os.path.join(proccessed_data_path,'dataset9.csv')
df = pd.read_csv(train_path)
labels=df['Churn']
x = df.drop(columns=['Churn','Unnamed: 0'],axis = 'columns')
y=np.ravel(labels)
oversample = SMOTENC(categorical_features=[10, 11,12,13])
x, y = oversample.fit_resample(x, y)
sc = RobustScaler()
x = pd.DataFrame(sc.fit_transform(x),columns = x.columns)
ii = 1

xgb_model = XGBClassifier(random_state=1,learning_rate=0.05, max_depth=7,eval_metric='mlogloss',use_label_encoder =False ,objective="binary:logistic")
dt_model=DecisionTreeClassifier(random_state=1,criterion='entropy',max_depth = 7,min_samples_leaf=30)  
nn_model = Sequential()
nn_model.add(Dense(64, kernel_regularizer=tf.keras.regularizers.l2(0.001), input_dim=14, activation='relu' ))
nn_model.add(Dropout(rate=0.2))
nn_model.add(Dense(8,kernel_regularizer=tf.keras.regularizers.l2(0.001),activation='relu'))
nn_model.add(Dropout(rate=0.1))
nn_model.add(Dense(1, activation='sigmoid'))

callback = tf.keras.callbacks.EarlyStopping(monitor='val_acc',patience=70,restore_best_weights=True)
cv = StratifiedKFold(n_splits=10)
for train_index, test_index in cv.split(x, y):
	print("Iteration" , ii)
	Xi_train, Xi_test = x.loc[train_index], x.loc[train_index]
	yi_train, yi_test = y[train_index], y[test_index]
	xgb_model.fit(Xi_train,yi_train)
	shap_values = shap.TreeExplainer(xgb_model).shap_values(Xi_test,approximate=True)
	#shap.summary_plot(shap_values, Xi_test, plot_type="bar")
	vals = np.abs(shap_values).mean(axis=0)
	print(pd.DataFrame(list(zip(vals)),columns=['XGB - feature_importance_vals']).to_string(index=False))    
	Xi_train, Xi_test = x.loc[train_index], x.loc[test_index]
	yi_train, yi_test = y[train_index], y[test_index]
	dt_model.fit(Xi_train,yi_train)
	explainer = shap.TreeExplainer(dt_model)
	shap_values = explainer.shap_values(Xi_test)
	#shap.summary_plot(shap_values[1], Xi_test, plot_type="bar")
	vals = np.abs(shap_values[1]).mean(axis=0)
	print( pd.DataFrame(list(zip(vals)),columns=['DT - feature_importance_vals']).to_string(index=False))
	lr_schedule= tf.keras.optimizers.schedules.InverseTimeDecay( 0.001,decay_steps=(Xi_train.shape[0]/32)*50,decay_rate=1,staircase=False)
	nn_model.compile(loss = "binary_crossentropy", optimizer = tf.keras.optimizers.Adam(lr_schedule), metrics=['accuracy'])
	nn_model.fit(Xi_train, yi_train, validation_data=(Xi_test, yi_test), epochs=150, batch_size=10,verbose=0,callbacks=[callback])
	explainer = shap.DeepExplainer(nn_model, Xi_train[1:100].to_numpy())
	shap_values = explainer.shap_values(Xi_test[1:20].to_numpy())
	vals = np.abs(shap_values[0]).mean(axis=0)
	print( pd.DataFrame(list(zip(vals)),columns=['NN - feature_importance_vals']).to_string(index=False))
	ii += 1                             

Instructions for updating:
non-resource variables are not supported in the long term
Iteration 1


ntree_limit is deprecated, use `iteration_range` or model slicing instead.


 XGB - feature_importance_vals
                      0.225223
                      0.000000
                      1.557905
                      0.578319
                      0.149443
                      0.192055
                      0.185146
                      0.210804
                      0.560530
                      0.092910
                      0.091699
                      0.015600
                      0.075954
                      0.229413
 DT - feature_importance_vals
                     0.000000
                     0.011981
                     0.355226
                     0.085613
                     0.000000
                     0.005824
                     0.009930
                     0.017179
                     0.066147
                     0.014085
                     0.000000
                     0.006282
                     0.004064
                     0.010377
Instructions for updating:
This property should not be used in TensorFlow 2.0, as upd

keras is no longer supported, please use tf.keras instead.


 NN - feature_importance_vals
                     0.036789
                     0.008164
                     0.150063
                     0.112156
                     0.017607
                     0.006386
                     0.020616
                     0.097577
                     0.127890
                     0.011623
                     0.000000
                     0.006813
                     0.043254
                     0.028244
Iteration 2


...

 XGB - feature_importance_vals
                      0.248235
                      0.000000
                      1.578202
                      0.601219
                      0.134656
                      0.205660
                      0.168208
                      0.228406
                      0.554241
                      0.121293
                      0.028002
                      0.047966
                      0.062152
                      0.157553
 DT - feature_importance_vals
                     0.000000
                     0.019328
                     0.358710
                     0.073775
                     0.013589
                     0.016767
                     0.008298
                     0.019507
                     0.048262
                     0.000000
                     0.000000
                     0.000000
                     0.000000
                     0.000000
 NN - feature_importance_vals
                     0.023605
                     0.00

ntree_limit is deprecated, use `iteration_range` or model slicing instead.


 XGB - feature_importance_vals
                      0.190491
                      0.000000
                      1.720529
                      0.580627
                      0.141454
                      0.219221
                      0.170637
                      0.278492
                      0.554586
                      0.082768
                      0.045676
                      0.032152
                      0.086437
                      0.259811
 DT - feature_importance_vals
                     0.000000
                     0.015470
                     0.227887
                     0.079875
                     0.013940
                     0.000000
                     0.010367
                     0.008698
                     0.089670
                     0.000000
                     0.002773
                     0.000000
                     0.012892
                     0.036315
 NN - feature_importance_vals
                     0.049445
                     0.01

ntree_limit is deprecated, use `iteration_range` or model slicing instead.


 XGB - feature_importance_vals
                      0.253494
                      0.000000
                      1.527292
                      0.701740
                      0.157068
                      0.213823
                      0.142969
                      0.290067
                      0.550576
                      0.082066
                      0.029910
                      0.016086
                      0.070116
                      0.157570
 DT - feature_importance_vals
                     0.000000
                     0.016119
                     0.271079
                     0.083885
                     0.000000
                     0.022884
                     0.012258
                     0.031242
                     0.082899
                     0.000000
                     0.000000
                     0.000000
                     0.000000
                     0.008506
 NN - feature_importance_vals
                     0.019328
                     0.00